# 1. K-means로 학습하기 (iris)

In [2]:
import pandas as pd
import numpy as np

In [3]:
# data load
iris=pd.DataFrame(['data','target'])
iris.data = pd.read_csv("dataset/Stars.csv")
iris.target = pd.DataFrame(iris.data.Type)
iris.data = iris.data.drop(['Color', 'Spectral_Class', 'Type'], axis=1)
iris.data.columns=[0,1,2,3]
iris.target.columns = [0]

iris.data

<ipython-input-3-023e802f52b9>:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  iris.data = pd.read_csv("dataset/Stars.csv")
<ipython-input-3-023e802f52b9>:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  iris.target = pd.DataFrame(iris.data.Type)


,0,1,2,3
0,3068,0.002400,0.1700,16.12
1,3042,0.000500,0.1542,16.60
2,2600,0.000300,0.1020,18.70
3,2800,0.000200,0.1600,16.65
4,1939,0.000138,0.1030,20.06
...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93
236,30839,834042.000000,1194.0000,-10.63
237,8829,537493.000000,1423.0000,-10.73
238,9235,404940.000000,1112.0000,-11.23


In [4]:
# 10번 반복하는 pipeline 함수 제작

from sklearn.pipeline import Pipeline

def pipe(model: Pipeline, train, target):
    result = []
    for i in range(0,10,1):
        model.fit(train, target)
        result.append(model.predict(train))
    return result

In [5]:
# normal_iris_kmeans accuracy

from sklearn.cluster import KMeans

pipeline = Pipeline([('Kmeans', KMeans(n_clusters=6, init='random'))]) # normal predict 생성
result_norm = pipe(pipeline, iris.data, iris.target)


from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score # Adjusted rand score는 음수가 나올수 있으므로 백분율 계산을 위해서 Normalized mutual information을 사용했다. 

score = []
for i in result_norm:
    score.append(normalized_mutual_info_score(iris.target[0], i))

print(np.mean(score))

0.4526822938224745


In [6]:
# PCA _iris_kemans accuracy

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

# 각 정규화 된 값들을 PCA로 차원축소 시키고 값이 어떻게 변화하는지 살펴본다.
# 일단 컴포넌트는 2로 하고 1~3중 어느값이 가장 좋은 결과를 가지는지 학습시켜본다.
pipeline_minmax = Pipeline([('MinMax', MinMaxScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_maxabs = Pipeline([('MaxAbs', MaxAbsScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_standard = Pipeline([('Standard', StandardScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_robust = Pipeline([('Robust', RobustScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_normalize = Pipeline([('Normalize', Normalizer()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])

result_minmax = pipe(pipeline_minmax, iris.data, iris.target)
result_maxabs = pipe(pipeline_maxabs, iris.data, iris.target)
result_standard = pipe(pipeline_standard, iris.data, iris.target)
result_robust = pipe(pipeline_robust, iris.data, iris.target)
result_normalize = pipe(pipeline_normalize, iris.data, iris.target)

score_minmax, score_maxabs, score_standard, score_robust, score_normalize = [], [], [], [], []
for i in range(0,10,1):
    score_minmax.append(normalized_mutual_info_score(iris.target[0], result_minmax[i]))
    score_maxabs.append(normalized_mutual_info_score(iris.target[0], result_maxabs[i]))
    score_standard.append(normalized_mutual_info_score(iris.target[0], result_standard[i]))
    score_robust.append(normalized_mutual_info_score(iris.target[0], result_robust[i]))
    score_normalize.append(normalized_mutual_info_score(iris.target[0], result_normalize[i]))

print('minmax_pca:',np.mean(score_minmax))
print('maxabs_pca:',np.mean(score_maxabs))
print('standard_pca:',np.mean(score_standard))
print('robust_pca:',np.mean(score_robust))
print('normalize_pca:',np.mean(score_normalize))

# star data는 -값을 가져  log scaling을 할 수 없다.
# normalizer의 accuracy가 가장 높으므로 maxabs로 전처리된 값을 이용해 PCA의 컴포넌트를 1~3까지 넣어본다.

minmax_pca: 0.652067752432415
maxabs_pca: 0.6673590929856366
standard_pca: 0.6146000856528693
robust_pca: 0.5676579450235579
normalize_pca: 0.50691656893753


In [7]:
# PCA _iris_kemans accuracy (change_component)

pipeline1 = Pipeline([('MaxAbs', MaxAbsScaler()),('PCA', PCA(n_components=1)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline2 = Pipeline([('MaxAbs', MaxAbsScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline3 = Pipeline([('MaxAbs', MaxAbsScaler()),('PCA', PCA(n_components=3)),('Kmeans', KMeans(n_clusters=6, init='random'))])

result1 = pipe(pipeline1, iris.data, iris.target)
result2 = pipe(pipeline2, iris.data, iris.target)
result3 = pipe(pipeline3, iris.data, iris.target)

score1, score2, score3 = [], [], []

for i in range(0,10,1):
    score1.append(normalized_mutual_info_score(iris.target[0], result1[i]))
    score2.append(normalized_mutual_info_score(iris.target[0], result2[i]))
    score3.append(normalized_mutual_info_score(iris.target[0], result3[i]))

print('component=1:',np.mean(score1))
print('component=2:',np.mean(score2))
print('component=3:',np.mean(score3))

# component는 1개일 때 가장 좋은 결과를 보였다. 다만 dataset 마다 결과는 다를 것으로보인다.

component=1: 0.7812379025086349
component=2: 0.6511613762232875
component=3: 0.6896281627376561


In [8]:
# TSNE는 predict가 없기 때문에 pipeline에 넣을 경우 오류가 생긴다. 따라서 이를 우회해줄 TSNE Handler 함수를 만든다. (https://data-newbie.tistory.com/568)

from sklearn.manifold import TSNE
from sklearn.base import BaseEstimator, TransformerMixin

class TSNEHandler(BaseEstimator, TransformerMixin):
    def __init__(self,n_components=2, random_state=0): # randomstate는 0으로 고정해서 결과값을 통일한다.
        self.tsne =TSNE(n_components=n_components, random_state = random_state, method="exact")
    
    def fit(self,x):
        self.tsne.fit(x)
        
    def transform(self,x):
        x = self.tsne.fit_transform(x)
        return x
        
    def fit_transform(self,x,y):
        x = self.tsne.fit_transform(x)    
        return x

In [9]:
# TSNE_iris_kmeans accuracy
# TSNE의 경우 코드 실행에 시간이 오래 걸리므로 한번의 학습만 실행

pipeline_minmax = Pipeline([('MinMax', MinMaxScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_maxabs = Pipeline([('MaxAbs', MaxAbsScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_standard = Pipeline([('Standard', StandardScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_robust = Pipeline([('Robust', RobustScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline_normalize = Pipeline([('Normalize', Normalizer()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])

result_minmax = pipeline_minmax.fit_predict(iris.data, iris.target)
result_maxabs = pipeline_maxabs.fit_predict(iris.data, iris.target)
result_standard = pipeline_standard.fit_predict(iris.data, iris.target)
result_robust = pipeline_robust.fit_predict(iris.data, iris.target)
result_normalize = pipeline_normalize.fit_predict(iris.data, iris.target)

score_minmax = normalized_mutual_info_score(iris.target[0], result_minmax)
score_maxabs = normalized_mutual_info_score(iris.target[0], result_maxabs)
score_standard = normalized_mutual_info_score(iris.target[0], result_standard)
score_robust = normalized_mutual_info_score(iris.target[0], result_robust)
score_normalize = normalized_mutual_info_score(iris.target[0], result_normalize)

print('minmax_TSNE:',score_minmax)
print('maxabs_TSNE:',score_maxabs)
print('standard_TSNE:',score_standard)
print('robust_TSNE:',score_robust)
print('normalize_TSNE:',score_normalize)

# 이번에는 robust의 accuracy가 가장 높다. robust로 전처리된 값을 이용해 TSNE의 컴포넌트를 1~3까지 넣어본다.

minmax_TSNE: 0.9335691044666057
maxabs_TSNE: 0.9415180988922823
standard_TSNE: 0.9386666362644679
robust_TSNE: 0.9514895154209181
normalize_TSNE: 0.6626632953103392


In [10]:
# TSNE_iris_kemans accuracy (change_component)

pipeline1 = Pipeline([('Robust', RobustScaler()),('t-SNE', TSNEHandler(n_components=1, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline2 = Pipeline([('Robust', RobustScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])
pipeline3 = Pipeline([('Robust', RobustScaler()),('t-SNE', TSNEHandler(n_components=3, random_state=0)),('Kmeans', KMeans(n_clusters=6, init='random'))])

result1 = pipeline1.fit_predict(iris.data, iris.target)
result2 = pipeline2.fit_predict(iris.data, iris.target)
result3 = pipeline3.fit_predict(iris.data, iris.target)

score1 = normalized_mutual_info_score(iris.target[0], result1)
score2 = normalized_mutual_info_score(iris.target[0], result2)
score3 = normalized_mutual_info_score(iris.target[0], result3)

print('component=1:',score1)
print('component=2:',score2)
print('component=3:',score3)

# 이번에는 PCA와 다르게 component가 2일 때 가장 좋은 결과를 보여줬다.

component=1: 0.8979791562696023
component=2: 0.951489515420918
component=3: 0.5108186254766569
